In [4]:
from absl import app, flags, logging
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

from typing import Any, Callable, Dict, List, Optional
import os

2021-09-09 00:31:01.666465: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-09 00:31:01.666522: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
tx = tf.constant(['abc def','xyz abc apple', 'abc xyz', 'appel pen'], dtype=tf.string)
ty = tf.constant([1, 1, 0, 0], dtype=tf.int64)
vx = tf.constant(['apple peer','abc apple', 'xyz', 'vvvv'], dtype=tf.string)
vy = tf.constant([1, 0, 0, 1], dtype=tf.int64)

2021-09-09 00:31:03.576178: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-09-09 00:31:03.576233: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-09-09 00:31:03.576262: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jk-tpu-node-1): /proc/driver/nvidia/version does not exist
2021-09-09 00:31:03.576757: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
PREPROCESS_URI = 'gs://tfhub-modules/tensorflow/bert_en_cased_preprocess/3/uncompressed'

def create_bert_preprocess_model(seq_length=128):

  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text_input')

  bert_preprocess = hub.load(PREPROCESS_URI)
  tokenizer = hub.KerasLayer(bert_preprocess.tokenize, name='tokenizer')
  packer = hub.KerasLayer(bert_preprocess.bert_pack_inputs,
                          arguments=dict(seq_length=seq_length),
                          name='packer')                                  
                                     
  tokenized_inputs = [tokenizer(segment) for segment in [text_input]]
  model_inputs = packer(tokenized_inputs)
                                     
  return tf.keras.Model(text_input, model_inputs)

In [9]:
preprocessor = create_bert_preprocess_model()

In [ ]:
for element in train_ds:
    print(element)

In [17]:
MODEL_URI = 'gs://tfhub-modules/tensorflow/bert_en_cased_L-24_H-1024_A-16/3/uncompressed'


def get_bert_model(seq_len=128):
    
    bert_layer = hub.KerasLayer(
        handle=MODEL_URI, 
        trainable=True,
        name='encoder'
    )
    encoder_inputs = dict(
        input_word_ids = tf.keras.layers.Input(shape=(seq_len,), dtype=tf.int32, name="input_word_ids"),
        input_mask = tf.keras.layers.Input(shape=(seq_len,), dtype=tf.int32, name="input_mask"),
        input_type_ids = tf.keras.layers.Input(shape=(seq_len,), dtype=tf.int32, name="segment_ids"),
    )
    encoder_outputs = bert_layer(encoder_inputs)
    embedding = encoder_outputs["pooled_output"]
    logits = tf.keras.layers.Dense(1)(embedding)
    softmax_prob = tf.keras.layers.Softmax()(logits)
    model = tf.keras.models.Model(inputs=encoder_inputs,
                                  outputs=softmax_prob)
    
    return model


In [18]:
def create_dataset(inputs, labels, batch_size, preprocessor):
  dataset = tf.data.Dataset.from_tensor_slices((inputs, labels))
  dataset = dataset.batch(batch_size)
  dataset = dataset.map(lambda x, y: (preprocessor(x), y))
  return dataset

In [19]:
train_ds = create_dataset(tx, ty, 2, preprocessor)
valid_ds = create_dataset(vx, vy, 2, preprocessor)

In [20]:
model = get_bert_model()

In [21]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_mask (InputLayer)         [(None, 128)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_word_ids (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
encoder (KerasLayer)            {'encoder_outputs':  333579265   input_mask[0][0]                 
                                                                 segment_ids[0][0]          

In [22]:
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss=tf.keras.losses.BinaryCrossentropy(), metrics=["accuracy"])

In [23]:
model.fit(x=train_ds, validation_data=valid_ds)

2/2 [==============================] - 63s 12s/step - loss: 7.6246 - accuracy: 0.5000 - val_loss: 7.6246 - val_accuracy: 0.5000
